In [1]:
import pandas as pd

In [2]:
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

In [3]:
df = pd.read_json("data/natural_question/v1.0-simplified_simplified-nq-train.jsonl", lines=True, nrows=100)

In [4]:
df.head()

,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018


In [5]:
df.iloc[3]['question_text']

'who had the most wins in the nfl'

In [6]:
df.iloc[3]['question_text']

'who had the most wins in the nfl'

In [7]:
df.iloc[3]['document_text'].split(" ")[512:514]

['Tom', 'Brady']

In [139]:
dev_df = pd.read_json("data/natural_question/v1.0-simplified_nq-dev-all.jsonl", lines=True, nrows=2000)

In [12]:
# dev_df.head()

In [13]:
dev_df.iloc[4]['question_text']

'star wars the clone wars anakin voice actor'

In [14]:
# dev_df.iloc[4]['annotations'] # 173-179

In [15]:
dev_df.iloc[4]

annotations               [{'annotation_id': 9697565976974065308, 'long_...
document_html             <!DOCTYPE html>\n<HTML class="client-js ve-not...
document_title                                                  Matt Lanter
document_tokens           [{'end_byte': 96, 'html_token': False, 'start_...
document_url              https://en.wikipedia.org//w/index.php?title=Ma...
example_id                                              8494342737238168262
long_answer_candidates    [{'end_byte': 57654, 'end_token': 79, 'start_b...
question_text                   star wars the clone wars anakin voice actor
question_tokens           [star, wars, the, clone, wars, anakin, voice, ...
Name: 4, dtype: object

In [19]:
!pip install protobuf
# !pip install sentencepiece

     |████████████████████████████████| 961 kB 1.5 MB/s eta 0:00:01


In [49]:
dev_df.iloc[150]['annotations'][1]


{'annotation_id': 5834450600727459786,
 'long_answer': {'candidate_index': 22,
  'end_byte': 197000,
  'end_token': 8433,
  'start_byte': 16013,
  'start_token': 684},
 'short_answers': [{'end_byte': 133179,
   'end_token': 5612,
   'start_byte': 133173,
   'start_token': 5611}],
 'yes_no_answer': 'NONE'}

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

t5_qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-large-ssm-nq")
t5_tok = AutoTokenizer.from_pretrained("google/t5-large-ssm-nq")


In [29]:
input_ids = t5_tok("When was Franklin D. Roosevelt born?", return_tensors="pt").input_ids
gen_output = t5_qa_model.generate(input_ids)[0]

print(t5_tok.decode(gen_output, skip_special_tokens=True))

December 26, 1892


In [54]:
input_ids = t5_tok("Is 2+2=4?", return_tensors="pt").input_ids
gen_output = t5_qa_model.generate(input_ids)[0]

print(t5_tok.decode(gen_output, skip_special_tokens=True))

4


In [165]:
dev_df.iloc[100]

annotations               [{'annotation_id': 18296758491112774267, 'long...
document_html             <!DOCTYPE html>\n<HTML class="client-js ve-not...
document_title                                              F Is for Family
document_tokens           [{'end_byte': 93, 'html_token': False, 'start_...
document_url              https://en.wikipedia.org//w/index.php?title=F_...
example_id                                             -2704708103393811482
long_answer_candidates    [{'end_byte': 60750, 'end_token': 344, 'start_...
question_text                           who voices randy in f is for family
question_tokens                [who, voices, randy, in, f, is, for, family]
Name: 100, dtype: object

In [180]:
dev_df.iloc[4]['question_text']

# ' '.join([token['token'] for token in dev_df.iloc[100]['document_tokens'][1110:1112]])

'star wars the clone wars anakin voice actor'

In [ ]:
short_answer_string()

In [186]:
tracker = []

def has_short_answer(annotations: list):
    if annotations:
        return sum([len(annotation['short_answers']) > 0 for annotation in annotations]) > 1
    else:
        False

def short_answer_string(sa_annotation, example):
    start_token = sa_annotation['start_token']
    end_token = sa_annotation['end_token']
    token_dicts = example['document_tokens'][start_token:end_token]
    return ' '.join([token_dict['token'] for token_dict in token_dicts])
        
def score_short_answer(prediction: str, example: dict):
    annotations = example['annotations']
    
    tracker_sa = []
    for annotation in annotations:
        short_answers = annotation['short_answers']
    
        exact_match = any([short_answer_string(answer, example).lower() == prediction.lower() for answer in short_answers])
        tracker_sa.append([short_answer_string(answer, example) for answer in short_answers])
            
        if exact_match:
            break
    
    tracker.append({
        "annotated_answers": tracker_sa,
        "prediction": prediction,
        "exact_match": exact_match,
        "question": example["question_text"],
        "document_url": example["document_url"]
       })
    
    if exact_match:
        return 1
    else:
        return 0

score_short_answer('T.j. miller', dev_df.iloc[100])
    
    

1

In [187]:
def predict_short_answer(example):    
    input_ids = t5_tok(example['question_text'], return_tensors="pt").input_ids
    gen_output = t5_qa_model.generate(input_ids)[0]
    prediction = t5_tok.decode(gen_output, skip_special_tokens=True)
        
    return score_short_answer(prediction, example)

predict_short_answer(dev_df.iloc[100])
    

1

In [188]:
def score_short_answers(examples, nrows):
    
    correct, total, num_skipped = 0, 0, 0
    
    for i in range(nrows):
        if i % 30 == 0:
            print(i)
        
        example = examples.iloc[i]
        if not has_short_answer(example['annotations']):
            num_skipped += 1
            continue
        
        result = predict_short_answer(example)
        total += 1
        correct += result
    try:
        accuracy = round(correct/total, 4)
    except Exception:
        accuracy = None
    
    print(f"total: {total} correct: {correct} EM: {accuracy}, num_skipped: {num_skipped}")
            

In [189]:
tracker = []
score_short_answers(dev_df, 2000)



0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
total: 834 correct: 213 EM: 0.2554, num_skipped: 1166


In [190]:
df = pd.DataFrame(tracker)

df.to_csv("results.csv", index=False)
